In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np
from itertools import product


In [2]:
def generate_policy_combinations():
    # SALT configurations with explicit joint/non-joint amounts and phase-out parameters
    salt_configs = [
        {
            "name": "0_cap",
            "cap_joint": 0,
            "cap_other": 0,
            "phase_out": False,
            "phase_out_rate": 0,
            "phase_out_threshold_joint": 0,
            "phase_out_threshold_other": 0,
        },
        {
            "name": "tcja_base",
            "cap_joint": 10_000,
            "cap_other": 10_000,
            "phase_out": False,
            "phase_out_rate": 0,
            "phase_out_threshold_joint": 0,
            "phase_out_threshold_other": 0,
        },
        {
            "name": "tcja_base_with_married_bonus",
            "cap_joint": 20_000,
            "cap_other": 10_000,
            "phase_out": False,
            "phase_out_rate": 0,
            "phase_out_threshold_joint": 0,
            "phase_out_threshold_other": 0,
        },
        {
            "name": "tcja_base_with_phaseout",
            "cap_joint": 10_000,
            "cap_other": 10_000,
            "phase_out": True,
            "phase_out_rate": 0.1,  # 10%
            "phase_out_threshold_joint": 400_000,
            "phase_out_threshold_other": 200_000,
        },
        {
            "name": "tcja_married_bonus_and_phase_out",
            "cap_joint": 20_000,
            "cap_other": 10_000,
            "phase_out": True,
            "phase_out_rate": 0.1,  # 10%
            "phase_out_threshold_joint": 400_000,
            "phase_out_threshold_other": 200_000,
        },
        {
            "name": "uncapped",
            "cap_joint": float("inf"),
            "cap_other": float("inf"),
            "phase_out": False,
            "phase_out_rate": 0,
            "phase_out_threshold_joint": 0,
            "phase_out_threshold_other": 0,
        },
    ]

    # AMT configurations with explicit joint/non-joint amounts for 2026
    amt_configs = [
        {
            "name": "repealed",
            "exemption_joint": float("inf"),
            "exemption_other": float("inf"),
            "phase_out_joint": 0,
            "phase_out_other": 0
        },
        {
            "name": "pre_tcja_ex_pre_tcja_po",
            "exemption_joint": 140_565,
            "exemption_other": 90_394,
            "phase_out_joint": 1_285_409,
            "phase_out_other": 642_705
        },
        {
            "name": "pre_tcja_ex_tcja_po",
            "exemption_joint": 140_565,
            "exemption_other": 90_394,
            "phase_out_joint": 209_000,
            "phase_out_other": 156_700
        },
        {
            "name": "tcja_ex_pre_tcja_po",
            "exemption_joint": 109_700,
            "exemption_other": 70_500,
            "phase_out_joint": 1_285_409,
            "phase_out_other": 642_705
        },
        {
            "name": "tcja_both",
            "exemption_joint": 109_700,
            "exemption_other": 70_500,
            "phase_out_joint": 209_000,
            "phase_out_other": 156_700
        },
    ]

    behavioral = [True, False]

    return list(product(salt_configs, amt_configs, behavioral))

In [3]:
def get_behavioral_reform():
    """Returns the behavioral response reform parameters"""
    return {
        "gov.simulation.labor_supply_responses.elasticities.income": {
            "2024-01-01.2100-12-31": -0.05
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
            "2024-01-01.2100-12-31": 0.31
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
            "2024-01-01.2100-12-31": 0.28
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
            "2024-01-01.2100-12-31": 0.27
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
            "2024-01-01.2100-12-31": 0.27
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
            "2024-01-01.2100-12-31": 0.25
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
            "2024-01-01.2100-12-31": 0.25
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
            "2024-01-01.2100-12-31": 0.22
        },
        "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
            "2024-01-01.2100-12-31": 0.27
        }
    }

In [4]:
def create_reform_dict(salt_config, amt_config, behavioral):
    """Create reform dictionary with optional behavioral responses"""
    reform_dict = {}
    
    # SALT caps
    reform_dict.update({
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.JOINT": {
            "2026-01-01.2100-12-31": salt_config["cap_joint"]
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": salt_config["cap_joint"]
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SEPARATE": {
            "2026-01-01.2100-12-31": salt_config["cap_joint"] / 2
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SINGLE": {
            "2026-01-01.2100-12-31": salt_config["cap_other"]
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": salt_config["cap_other"]
        },
        # Add phase-out parameters
        "gov.contrib.salt_phase_out.in_effect": {
            "2026-01-01.2100-12-31": salt_config["phase_out"]
        },
        "gov.contrib.salt_phase_out.rate.other[1].rate": {
            "2026-01-01.2100-12-31": salt_config["phase_out_rate"] if salt_config["phase_out"] else 0
        },
        "gov.contrib.salt_phase_out.rate.joint[1].threshold": {
            "2026-01-01.2100-12-31": salt_config["phase_out_threshold_joint"] if salt_config["phase_out"] else 0
        },
        "gov.contrib.salt_phase_out.rate.other[1].threshold": {
            "2026-01-01.2100-12-31": salt_config["phase_out_threshold_other"] if salt_config["phase_out"] else 0
        }
    })

    # AMT parameters - only if not repealed
    reform_dict.update({
        "gov.irs.income.amt.exemption.amount.JOINT": {
            "2026-01-01.2026-12-31": amt_config["exemption_joint"]
        },
        "gov.irs.income.amt.exemption.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": amt_config["exemption_joint"]
        },
        "gov.irs.income.amt.exemption.amount.SEPARATE": {
            "2026-01-01.2026-12-31": amt_config["exemption_joint"] / 2
        },
        "gov.irs.income.amt.exemption.amount.SINGLE": {
            "2026-01-01.2026-12-31": amt_config["exemption_other"]
        },
        "gov.irs.income.amt.exemption.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": amt_config["exemption_other"]
        },
        # Phase-out thresholds
        "gov.irs.income.amt.exemption.phase_out.start.JOINT": {
            "2026-01-01.2026-12-31": amt_config["phase_out_joint"]
        },
        "gov.irs.income.amt.exemption.phase_out.start.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": amt_config["phase_out_joint"]
        },
        "gov.irs.income.amt.exemption.phase_out.start.SEPARATE": {
            "2026-01-01.2026-12-31": amt_config["phase_out_joint"] / 2
        },
        "gov.irs.income.amt.exemption.phase_out.start.SINGLE": {
            "2026-01-01.2026-12-31": amt_config["phase_out_other"]
        },
        "gov.irs.income.amt.exemption.phase_out.start.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": amt_config["phase_out_other"]
        }
    })

    # Add behavioral responses if enabled
    if behavioral:
        reform_dict.update(get_behavioral_reform())
    
    return reform_dict



In [5]:
def get_tcja_extension_reform():
    """Returns the reform dictionary for TCJA extension"""
    return {
        "gov.irs.credits.ctc.amount.base[0].amount": {
            "2026-01-01.2100-12-31": 2000
        },
        "gov.irs.credits.ctc.phase_out.threshold.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 200000
        },
        "gov.irs.credits.ctc.phase_out.threshold.JOINT": {
            "2026-01-01.2100-12-31": 400000
        },
        "gov.irs.credits.ctc.phase_out.threshold.SEPARATE": {
            "2026-01-01.2100-12-31": 200000
        },
        "gov.irs.credits.ctc.phase_out.threshold.SINGLE": {
            "2026-01-01.2100-12-31": 200000
        },
        "gov.irs.credits.ctc.phase_out.threshold.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 400000
        },
        "gov.irs.credits.ctc.refundable.individual_max": {
            "2026-01-01.2026-12-31": 1800,
            "2027-01-01.2027-12-31": 1800,
            "2028-01-01.2028-12-31": 1800,
            "2029-01-01.2029-12-31": 1900,
            "2030-01-01.2030-12-31": 1900,
            "2031-01-01.2031-12-31": 1900,
            "2032-01-01.2032-12-31": 2000,
            "2033-01-01.2033-12-31": 2000,
            "2034-01-01.2034-12-31": 2000,
            "2035-01-01.2035-12-31": 2000
        },
        "gov.irs.credits.ctc.refundable.phase_in.threshold": {
            "2026-01-01.2100-12-31": 2500
        },
        "gov.irs.deductions.itemized.casualty.active": {
            "2026-01-01.2100-12-31": False
        },
        "gov.irs.deductions.itemized.charity.ceiling.all": {
            "2026-01-01.2100-12-31": 0.6
        },
        "gov.irs.deductions.itemized.limitation.agi_rate": {
            "2026-01-01.2026-12-31": 1,
            "2027-01-01.2027-12-31": 1,
            "2028-01-01.2028-12-31": 1,
            "2029-01-01.2029-12-31": 1,
            "2030-01-01.2030-12-31": 1,
            "2031-01-01.2031-12-31": 1,
            "2032-01-01.2032-12-31": 1,
            "2033-01-01.2033-12-31": 1,
            "2034-01-01.2034-12-31": 1
        },
        "gov.irs.deductions.itemized.limitation.applicable_amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 1000000,
            "2027-01-01.2027-12-31": 1000000,
            "2028-01-01.2028-12-31": 1000000,
            "2029-01-01.2029-12-31": 1000000,
            "2030-01-01.2030-12-31": 1000000,
            "2031-01-01.2031-12-31": 1000000,
            "2032-01-01.2032-12-31": 1000000,
            "2033-01-01.2033-12-31": 1000000,
            "2034-01-01.2034-12-31": 1000000
        },
        "gov.irs.deductions.itemized.limitation.applicable_amount.JOINT": {
            "2026-01-01.2026-12-31": 1000000,
            "2027-01-01.2027-12-31": 1000000,
            "2028-01-01.2028-12-31": 1000000,
            "2029-01-01.2029-12-31": 1000000,
            "2030-01-01.2030-12-31": 1000000,
            "2031-01-01.2031-12-31": 1000000,
            "2032-01-01.2032-12-31": 1000000,
            "2033-01-01.2033-12-31": 1000000,
            "2034-01-01.2034-12-31": 1000000
        },
        "gov.irs.deductions.itemized.limitation.applicable_amount.SEPARATE": {
            "2026-01-01.2026-12-31": 1000000,
            "2027-01-01.2027-12-31": 1000000,
            "2028-01-01.2028-12-31": 1000000,
            "2029-01-01.2029-12-31": 1000000,
            "2030-01-01.2030-12-31": 1000000,
            "2031-01-01.2031-12-31": 1000000,
            "2032-01-01.2032-12-31": 1000000,
            "2033-01-01.2033-12-31": 1000000,
            "2034-01-01.2034-12-31": 1000000
        },
        "gov.irs.deductions.itemized.limitation.applicable_amount.SINGLE": {
            "2026-01-01.2026-12-31": 1000000,
            "2027-01-01.2027-12-31": 1000000,
            "2028-01-01.2028-12-31": 1000000,
            "2029-01-01.2029-12-31": 1000000,
            "2030-01-01.2030-12-31": 1000000,
            "2031-01-01.2031-12-31": 1000000,
            "2032-01-01.2032-12-31": 1000000,
            "2033-01-01.2033-12-31": 1000000,
            "2034-01-01.2034-12-31": 1000000
        },
        "gov.irs.deductions.itemized.limitation.applicable_amount.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 1000000,
            "2027-01-01.2027-12-31": 1000000,
            "2028-01-01.2028-12-31": 1000000,
            "2029-01-01.2029-12-31": 1000000,
            "2030-01-01.2030-12-31": 1000000,
            "2031-01-01.2031-12-31": 1000000,
            "2032-01-01.2032-12-31": 1000000,
            "2033-01-01.2033-12-31": 1000000,
            "2034-01-01.2034-12-31": 1000000
        },
        "gov.irs.deductions.itemized.limitation.itemized_deduction_rate": {
            "2026-01-01.2026-12-31": 1,
            "2027-01-01.2027-12-31": 1,
            "2028-01-01.2028-12-31": 1,
            "2029-01-01.2029-12-31": 1,
            "2030-01-01.2030-12-31": 1,
            "2031-01-01.2031-12-31": 1,
            "2032-01-01.2032-12-31": 1,
            "2033-01-01.2033-12-31": 1,
            "2034-01-01.2034-12-31": 1
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 10000
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.JOINT": {
            "2026-01-01.2100-12-31": 10000
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SEPARATE": {
            "2026-01-01.2100-12-31": 5000
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SINGLE": {
            "2026-01-01.2100-12-31": 10000
        },
        "gov.irs.deductions.itemized.salt_and_real_estate.cap.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 10000
        },
        "gov.irs.deductions.qbi.max.business_property.rate": {
            "2026-01-01.2100-12-31": 0.025
        },
        "gov.irs.deductions.qbi.max.rate": {
            "2026-01-01.2100-12-31": 0.2
        },
        "gov.irs.deductions.qbi.max.w2_wages.alt_rate": {
            "2026-01-01.2100-12-31": 0.25
        },
        "gov.irs.deductions.qbi.max.w2_wages.rate": {
            "2026-01-01.2100-12-31": 0.5
        },
        "gov.irs.deductions.qbi.phase_out.length.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 50000
        },
        "gov.irs.deductions.qbi.phase_out.length.JOINT": {
            "2026-01-01.2100-12-31": 100000
        },
        "gov.irs.deductions.qbi.phase_out.length.SEPARATE": {
            "2026-01-01.2100-12-31": 50000
        },
        "gov.irs.deductions.qbi.phase_out.length.SINGLE": {
            "2026-01-01.2100-12-31": 50000
        },
        "gov.irs.deductions.qbi.phase_out.length.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 100000
        },
        "gov.irs.deductions.qbi.phase_out.start.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2024-12-31": 198225,
            "2025-01-01.2025-12-31": 200275,
            "2026-01-01.2026-12-31": 204900,
            "2027-01-01.2027-12-31": 209050,
            "2028-01-01.2028-12-31": 213075,
            "2029-01-01.2029-12-31": 217125,
            "2030-01-01.2030-12-31": 221375,
            "2031-01-01.2031-12-31": 225775,
            "2032-01-01.2032-12-31": 230275,
            "2033-01-01.2033-12-31": 234875,
            "2034-01-01.2034-12-31": 239600,
            "2035-01-01.2035-12-31": 244450
        },
        "gov.irs.deductions.qbi.phase_out.start.JOINT": {
            "2024-01-01.2024-12-31": 396450,
            "2025-01-01.2025-12-31": 400575,
            "2026-01-01.2026-12-31": 409800,
            "2027-01-01.2027-12-31": 418100,
            "2028-01-01.2028-12-31": 426175,
            "2029-01-01.2029-12-31": 434225,
            "2030-01-01.2030-12-31": 442775,
            "2031-01-01.2031-12-31": 451525,
            "2032-01-01.2032-12-31": 460525,
            "2033-01-01.2033-12-31": 469750,
            "2034-01-01.2034-12-31": 479200,
            "2035-01-01.2035-12-31": 488900
        },
        "gov.irs.deductions.qbi.phase_out.start.SEPARATE": {
            "2024-01-01.2024-12-31": 198225,
            "2025-01-01.2025-12-31": 200275,
            "2026-01-01.2026-12-31": 204900,
            "2027-01-01.2027-12-31": 209050,
            "2028-01-01.2028-12-31": 213075,
            "2029-01-01.2029-12-31": 217125,
            "2030-01-01.2030-12-31": 221375,
            "2031-01-01.2031-12-31": 225775,
            "2032-01-01.2032-12-31": 230275,
            "2033-01-01.2033-12-31": 234875,
            "2034-01-01.2034-12-31": 239600,
            "2035-01-01.2035-12-31": 244450
        },
        "gov.irs.deductions.qbi.phase_out.start.SINGLE": {
            "2024-01-01.2024-12-31": 198225,
            "2025-01-01.2025-12-31": 200275,
            "2026-01-01.2026-12-31": 204900,
            "2027-01-01.2027-12-31": 209050,
            "2028-01-01.2028-12-31": 213075,
            "2029-01-01.2029-12-31": 217125,
            "2030-01-01.2030-12-31": 221375,
            "2031-01-01.2031-12-31": 225775,
            "2032-01-01.2032-12-31": 230275,
            "2033-01-01.2033-12-31": 234875,
            "2034-01-01.2034-12-31": 239600,
            "2035-01-01.2035-12-31": 244450
        },
        "gov.irs.deductions.qbi.phase_out.start.SURVIVING_SPOUSE": {
            "2024-01-01.2024-12-31": 396450,
            "2025-01-01.2025-12-31": 400575,
            "2026-01-01.2026-12-31": 409800,
            "2027-01-01.2027-12-31": 418100,
            "2028-01-01.2028-12-31": 426175,
            "2029-01-01.2029-12-31": 434225,
            "2030-01-01.2030-12-31": 442775,
            "2031-01-01.2031-12-31": 451525,
            "2032-01-01.2032-12-31": 460525,
            "2033-01-01.2033-12-31": 469750,
            "2034-01-01.2034-12-31": 479200,
            "2035-01-01.2035-12-31": 488900
        },
        "gov.irs.deductions.standard.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 22950,
            "2027-01-01.2027-12-31": 23425,
            "2028-01-01.2028-12-31": 23875,
            "2029-01-01.2029-12-31": 24325,
            "2030-01-01.2030-12-31": 24800,
            "2031-01-01.2031-12-31": 25300,
            "2032-01-01.2032-12-31": 25800,
            "2033-01-01.2033-12-31": 26300,
            "2034-01-01.2034-12-31": 26825,
            "2035-01-01.2035-12-31": 27375
        },
        "gov.irs.deductions.standard.amount.JOINT": {
            "2026-01-01.2026-12-31": 30600,
            "2027-01-01.2027-12-31": 31225,
            "2028-01-01.2028-12-31": 31825,
            "2029-01-01.2029-12-31": 32425,
            "2030-01-01.2030-12-31": 33050,
            "2031-01-01.2031-12-31": 33725,
            "2032-01-01.2032-12-31": 34400,
            "2033-01-01.2033-12-31": 35075,
            "2034-01-01.2034-12-31": 35775,
            "2035-01-01.2035-12-31": 36500
        },
        "gov.irs.deductions.standard.amount.SEPARATE": {
            "2026-01-01.2026-12-31": 15300,
            "2027-01-01.2027-12-31": 15600,
            "2028-01-01.2028-12-31": 15900,
            "2029-01-01.2029-12-31": 16225,
            "2030-01-01.2030-12-31": 16525,
            "2031-01-01.2031-12-31": 16850,
            "2032-01-01.2032-12-31": 17200,
            "2033-01-01.2033-12-31": 17550,
            "2034-01-01.2034-12-31": 17900,
            "2035-01-01.2035-12-31": 18250
        },
        "gov.irs.deductions.standard.amount.SINGLE": {
            "2026-01-01.2026-12-31": 15300,
            "2027-01-01.2027-12-31": 15600,
            "2028-01-01.2028-12-31": 15900,
            "2029-01-01.2029-12-31": 16225,
            "2030-01-01.2030-12-31": 16525,
            "2031-01-01.2031-12-31": 16850,
            "2032-01-01.2032-12-31": 17200,
            "2033-01-01.2033-12-31": 17550,
            "2034-01-01.2034-12-31": 17900,
            "2035-01-01.2035-12-31": 18250
        },
        "gov.irs.deductions.standard.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 30600,
            "2027-01-01.2027-12-31": 31225,
            "2028-01-01.2028-12-31": 31825,
            "2029-01-01.2029-12-31": 32425,
            "2030-01-01.2030-12-31": 33050,
            "2031-01-01.2031-12-31": 33725,
            "2032-01-01.2032-12-31": 34400,
            "2033-01-01.2033-12-31": 35075,
            "2034-01-01.2034-12-31": 35775,
            "2035-01-01.2035-12-31": 36500
        },
        "gov.irs.income.amt.exemption.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 89925,
            "2027-01-01.2027-12-31": 91750,
            "2028-01-01.2028-12-31": 93525,
            "2029-01-01.2029-12-31": 95300,
            "2030-01-01.2030-12-31": 97150,
            "2031-01-01.2031-12-31": 99075,
            "2032-01-01.2032-12-31": 101050,
            "2033-01-01.2033-12-31": 103075,
            "2034-01-01.2034-12-31": 105150,
            "2035-01-01.2035-12-31": 107275
        },
        "gov.irs.income.amt.exemption.amount.JOINT": {
            "2026-01-01.2026-12-31": 139850,
            "2027-01-01.2027-12-31": 142675,
            "2028-01-01.2028-12-31": 145425,
            "2029-01-01.2029-12-31": 148200,
            "2030-01-01.2030-12-31": 151100,
            "2031-01-01.2031-12-31": 154100,
            "2032-01-01.2032-12-31": 157150,
            "2033-01-01.2033-12-31": 160300,
            "2034-01-01.2034-12-31": 163525,
            "2035-01-01.2035-12-31": 166850
        },
        "gov.irs.income.amt.exemption.amount.SEPARATE": {
            "2026-01-01.2026-12-31": 69925,
            "2027-01-01.2027-12-31": 71350,
            "2028-01-01.2028-12-31": 72725,
            "2029-01-01.2029-12-31": 74100,
            "2030-01-01.2030-12-31": 75550,
            "2031-01-01.2031-12-31": 77050,
            "2032-01-01.2032-12-31": 78575,
            "2033-01-01.2033-12-31": 80150,
            "2034-01-01.2034-12-31": 81775,
            "2035-01-01.2035-12-31": 83425
        },
        "gov.irs.income.amt.exemption.amount.SINGLE": {
            "2026-01-01.2026-12-31": 89925,
            "2027-01-01.2027-12-31": 91750,
            "2028-01-01.2028-12-31": 93525,
            "2029-01-01.2029-12-31": 95300,
            "2030-01-01.2030-12-31": 97150,
            "2031-01-01.2031-12-31": 99075,
            "2032-01-01.2032-12-31": 101050,
            "2033-01-01.2033-12-31": 103075,
            "2034-01-01.2034-12-31": 105150,
            "2035-01-01.2035-12-31": 107275
        },
        "gov.irs.income.amt.exemption.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 139850,
            "2027-01-01.2027-12-31": 142675,
            "2028-01-01.2028-12-31": 145425,
            "2029-01-01.2029-12-31": 148200,
            "2030-01-01.2030-12-31": 151100,
            "2031-01-01.2031-12-31": 154100,
            "2032-01-01.2032-12-31": 157150,
            "2033-01-01.2033-12-31": 160300,
            "2034-01-01.2034-12-31": 163525,
            "2035-01-01.2035-12-31": 166850
        },
        "gov.irs.income.amt.exemption.phase_out.start.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 639300,
            "2027-01-01.2027-12-31": 652250,
            "2028-01-01.2028-12-31": 664825,
            "2029-01-01.2029-12-31": 677425,
            "2030-01-01.2030-12-31": 690725,
            "2031-01-01.2031-12-31": 704400,
            "2032-01-01.2032-12-31": 718425,
            "2033-01-01.2033-12-31": 732825,
            "2034-01-01.2034-12-31": 747575,
            "2035-01-01.2035-12-31": 762675
        },
        "gov.irs.income.amt.exemption.phase_out.start.JOINT": {
            "2026-01-01.2026-12-31": 1278575,
            "2027-01-01.2027-12-31": 1304475,
            "2028-01-01.2028-12-31": 1329675,
            "2029-01-01.2029-12-31": 1354850,
            "2030-01-01.2030-12-31": 1381475,
            "2031-01-01.2031-12-31": 1408825,
            "2032-01-01.2032-12-31": 1436875,
            "2033-01-01.2033-12-31": 1465650,
            "2034-01-01.2034-12-31": 1495150,
            "2035-01-01.2035-12-31": 1525375
        },
        "gov.irs.income.amt.exemption.phase_out.start.SEPARATE": {
            "2026-01-01.2026-12-31": 639300,
            "2027-01-01.2027-12-31": 652250,
            "2028-01-01.2028-12-31": 664825,
            "2029-01-01.2029-12-31": 677425,
            "2030-01-01.2030-12-31": 690725,
            "2031-01-01.2031-12-31": 704400,
            "2032-01-01.2032-12-31": 718425,
            "2033-01-01.2033-12-31": 732825,
            "2034-01-01.2034-12-31": 747575,
            "2035-01-01.2035-12-31": 762675
        },
        "gov.irs.income.amt.exemption.phase_out.start.SINGLE": {
            "2026-01-01.2026-12-31": 639300,
            "2027-01-01.2027-12-31": 652250,
            "2028-01-01.2028-12-31": 664825,
            "2029-01-01.2029-12-31": 677425,
            "2030-01-01.2030-12-31": 690725,
            "2031-01-01.2031-12-31": 704400,
            "2032-01-01.2032-12-31": 718425,
            "2033-01-01.2033-12-31": 732825,
            "2034-01-01.2034-12-31": 747575,
            "2035-01-01.2035-12-31": 762675
        },
        "gov.irs.income.amt.exemption.phase_out.start.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 1278575,
            "2027-01-01.2027-12-31": 1304475,
            "2028-01-01.2028-12-31": 1329675,
            "2029-01-01.2029-12-31": 1354850,
            "2030-01-01.2030-12-31": 1381475,
            "2031-01-01.2031-12-31": 1408825,
            "2032-01-01.2032-12-31": 1436875,
            "2033-01-01.2033-12-31": 1465650,
            "2034-01-01.2034-12-31": 1495150,
            "2035-01-01.2035-12-31": 1525375
        },
        "gov.irs.income.bracket.rates.2": {
            "2026-01-01.2100-12-31": 0.12
        },
        "gov.irs.income.bracket.rates.3": {
            "2026-01-01.2100-12-31": 0.22
        },
        "gov.irs.income.bracket.rates.4": {
            "2026-01-01.2100-12-31": 0.24
        },
        "gov.irs.income.bracket.rates.5": {
            "2026-01-01.2100-12-31": 0.32
        },
        "gov.irs.income.bracket.rates.7": {
            "2026-01-01.2100-12-31": 0.37
        },
        "gov.irs.income.bracket.thresholds.3.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 105475,
            "2027-01-01.2027-12-31": 107600,
            "2028-01-01.2028-12-31": 109700,
            "2029-01-01.2029-12-31": 111775,
            "2030-01-01.2030-12-31": 113950,
            "2031-01-01.2031-12-31": 116225,
            "2032-01-01.2032-12-31": 118525,
            "2033-01-01.2033-12-31": 120900,
            "2034-01-01.2034-12-31": 123350,
            "2035-01-01.2035-12-31": 125825
        },
        "gov.irs.income.bracket.thresholds.3.JOINT": {
            "2026-01-01.2026-12-31": 210950,
            "2027-01-01.2027-12-31": 215225,
            "2028-01-01.2028-12-31": 219375,
            "2029-01-01.2029-12-31": 223525,
            "2030-01-01.2030-12-31": 227925,
            "2031-01-01.2031-12-31": 232425,
            "2032-01-01.2032-12-31": 237075,
            "2033-01-01.2033-12-31": 241825,
            "2034-01-01.2034-12-31": 246675,
            "2035-01-01.2035-12-31": 251675
        },
        "gov.irs.income.bracket.thresholds.3.SEPARATE": {
            "2026-01-01.2026-12-31": 105475,
            "2027-01-01.2027-12-31": 107600,
            "2028-01-01.2028-12-31": 109700,
            "2029-01-01.2029-12-31": 111775,
            "2030-01-01.2030-12-31": 113950,
            "2031-01-01.2031-12-31": 116225,
            "2032-01-01.2032-12-31": 118525,
            "2033-01-01.2033-12-31": 120900,
            "2034-01-01.2034-12-31": 123350,
            "2035-01-01.2035-12-31": 125825
        },
        "gov.irs.income.bracket.thresholds.3.SINGLE": {
            "2026-01-01.2026-12-31": 105475,
            "2027-01-01.2027-12-31": 107600,
            "2028-01-01.2028-12-31": 109700,
            "2029-01-01.2029-12-31": 111775,
            "2030-01-01.2030-12-31": 113950,
            "2031-01-01.2031-12-31": 116225,
            "2032-01-01.2032-12-31": 118525,
            "2033-01-01.2033-12-31": 120900,
            "2034-01-01.2034-12-31": 123350,
            "2035-01-01.2035-12-31": 125825
        },
        "gov.irs.income.bracket.thresholds.3.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 210950,
            "2027-01-01.2027-12-31": 215225,
            "2028-01-01.2028-12-31": 219375,
            "2029-01-01.2029-12-31": 223525,
            "2030-01-01.2030-12-31": 227925,
            "2031-01-01.2031-12-31": 232425,
            "2032-01-01.2032-12-31": 237075,
            "2033-01-01.2033-12-31": 241825,
            "2034-01-01.2034-12-31": 246675,
            "2035-01-01.2035-12-31": 251675
        },
        "gov.irs.income.bracket.thresholds.4.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 201350,
            "2027-01-01.2027-12-31": 205425,
            "2028-01-01.2028-12-31": 209400,
            "2029-01-01.2029-12-31": 213375,
            "2030-01-01.2030-12-31": 217550,
            "2031-01-01.2031-12-31": 221875,
            "2032-01-01.2032-12-31": 226275,
            "2033-01-01.2033-12-31": 230825,
            "2034-01-01.2034-12-31": 235475,
            "2035-01-01.2035-12-31": 240225
        },
        "gov.irs.income.bracket.thresholds.4.JOINT": {
            "2026-01-01.2026-12-31": 402725,
            "2027-01-01.2027-12-31": 410875,
            "2028-01-01.2028-12-31": 418800,
            "2029-01-01.2029-12-31": 426725,
            "2030-01-01.2030-12-31": 435125,
            "2031-01-01.2031-12-31": 443725,
            "2032-01-01.2032-12-31": 452575,
            "2033-01-01.2033-12-31": 461650,
            "2034-01-01.2034-12-31": 470925,
            "2035-01-01.2035-12-31": 480450
        },
        "gov.irs.income.bracket.thresholds.4.SEPARATE": {
            "2026-01-01.2026-12-31": 201350,
            "2027-01-01.2027-12-31": 205425,
            "2028-01-01.2028-12-31": 209400,
            "2029-01-01.2029-12-31": 213375,
            "2030-01-01.2030-12-31": 217550,
            "2031-01-01.2031-12-31": 221875,
            "2032-01-01.2032-12-31": 226275,
            "2033-01-01.2033-12-31": 230825,
            "2034-01-01.2034-12-31": 235475,
            "2035-01-01.2035-12-31": 240225
        },
        "gov.irs.income.bracket.thresholds.4.SINGLE": {
            "2026-01-01.2026-12-31": 201350,
            "2027-01-01.2027-12-31": 205425,
            "2028-01-01.2028-12-31": 209400,
            "2029-01-01.2029-12-31": 213375,
            "2030-01-01.2030-12-31": 217550,
            "2031-01-01.2031-12-31": 221875,
            "2032-01-01.2032-12-31": 226275,
            "2033-01-01.2033-12-31": 230825,
            "2034-01-01.2034-12-31": 235475,
            "2035-01-01.2035-12-31": 240225
        },
        "gov.irs.income.bracket.thresholds.4.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 402725,
            "2027-01-01.2027-12-31": 410875,
            "2028-01-01.2028-12-31": 418800,
            "2029-01-01.2029-12-31": 426725,
            "2030-01-01.2030-12-31": 435125,
            "2031-01-01.2031-12-31": 443725,
            "2032-01-01.2032-12-31": 452575,
            "2033-01-01.2033-12-31": 461650,
            "2034-01-01.2034-12-31": 470925,
            "2035-01-01.2035-12-31": 480450
        },
        "gov.irs.income.bracket.thresholds.5.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 255700,
            "2027-01-01.2027-12-31": 260875,
            "2028-01-01.2028-12-31": 265925,
            "2029-01-01.2029-12-31": 270950,
            "2030-01-01.2030-12-31": 276275,
            "2031-01-01.2031-12-31": 281750,
            "2032-01-01.2032-12-31": 287375,
            "2033-01-01.2033-12-31": 293125,
            "2034-01-01.2034-12-31": 299025,
            "2035-01-01.2035-12-31": 305075
        },
        "gov.irs.income.bracket.thresholds.5.JOINT": {
            "2026-01-01.2026-12-31": 511400,
            "2027-01-01.2027-12-31": 521775,
            "2028-01-01.2028-12-31": 531850,
            "2029-01-01.2029-12-31": 541925,
            "2030-01-01.2030-12-31": 552575,
            "2031-01-01.2031-12-31": 563500,
            "2032-01-01.2032-12-31": 574725,
            "2033-01-01.2033-12-31": 586250,
            "2034-01-01.2034-12-31": 598050,
            "2035-01-01.2035-12-31": 610125
        },
        "gov.irs.income.bracket.thresholds.5.SEPARATE": {
            "2026-01-01.2026-12-31": 255700,
            "2027-01-01.2027-12-31": 260875,
            "2028-01-01.2028-12-31": 265925,
            "2029-01-01.2029-12-31": 270950,
            "2030-01-01.2030-12-31": 276275,
            "2031-01-01.2031-12-31": 281750,
            "2032-01-01.2032-12-31": 287375,
            "2033-01-01.2033-12-31": 293125,
            "2034-01-01.2034-12-31": 299025,
            "2035-01-01.2035-12-31": 305075
        },
        "gov.irs.income.bracket.thresholds.5.SINGLE": {
            "2026-01-01.2026-12-31": 255700,
            "2027-01-01.2027-12-31": 260875,
            "2028-01-01.2028-12-31": 265925,
            "2029-01-01.2029-12-31": 270950,
            "2030-01-01.2030-12-31": 276275,
            "2031-01-01.2031-12-31": 281750,
            "2032-01-01.2032-12-31": 287375,
            "2033-01-01.2033-12-31": 293125,
            "2034-01-01.2034-12-31": 299025,
            "2035-01-01.2035-12-31": 305075
        },
        "gov.irs.income.bracket.thresholds.5.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 511400,
            "2027-01-01.2027-12-31": 521775,
            "2028-01-01.2028-12-31": 531850,
            "2029-01-01.2029-12-31": 541925,
            "2030-01-01.2030-12-31": 552575,
            "2031-01-01.2031-12-31": 563500,
            "2032-01-01.2032-12-31": 574725,
            "2033-01-01.2033-12-31": 586250,
            "2034-01-01.2034-12-31": 598050,
            "2035-01-01.2035-12-31": 610125
        },
        "gov.irs.income.bracket.thresholds.6.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2026-12-31": 639300,
            "2027-01-01.2027-12-31": 652250,
            "2028-01-01.2028-12-31": 664825,
            "2029-01-01.2029-12-31": 677425,
            "2030-01-01.2030-12-31": 690725,
            "2031-01-01.2031-12-31": 704400,
            "2032-01-01.2032-12-31": 718425,
            "2033-01-01.2033-12-31": 732825,
            "2034-01-01.2034-12-31": 747575,
            "2035-01-01.2035-12-31": 762675
        },
        "gov.irs.income.bracket.thresholds.6.JOINT": {
            "2026-01-01.2026-12-31": 767125,
            "2027-01-01.2027-12-31": 782650,
            "2028-01-01.2028-12-31": 797775,
            "2029-01-01.2029-12-31": 812875,
            "2030-01-01.2030-12-31": 828850,
            "2031-01-01.2031-12-31": 845250,
            "2032-01-01.2032-12-31": 862100,
            "2033-01-01.2033-12-31": 879350,
            "2034-01-01.2034-12-31": 897050,
            "2035-01-01.2035-12-31": 915200
        },
        "gov.irs.income.bracket.thresholds.6.SEPARATE": {
            "2026-01-01.2026-12-31": 383550,
            "2027-01-01.2027-12-31": 391325,
            "2028-01-01.2028-12-31": 398875,
            "2029-01-01.2029-12-31": 406450,
            "2030-01-01.2030-12-31": 414425,
            "2031-01-01.2031-12-31": 422625,
            "2032-01-01.2032-12-31": 431050,
            "2033-01-01.2033-12-31": 439675,
            "2034-01-01.2034-12-31": 448525,
            "2035-01-01.2035-12-31": 457600
        },
        "gov.irs.income.bracket.thresholds.6.SINGLE": {
            "2026-01-01.2026-12-31": 639300,
            "2027-01-01.2027-12-31": 652250,
            "2028-01-01.2028-12-31": 664825,
            "2029-01-01.2029-12-31": 677425,
            "2030-01-01.2030-12-31": 690725,
            "2031-01-01.2031-12-31": 704400,
            "2032-01-01.2032-12-31": 718425,
            "2033-01-01.2033-12-31": 732825,
            "2034-01-01.2034-12-31": 747575,
            "2035-01-01.2035-12-31": 762675
        },
        "gov.irs.income.bracket.thresholds.6.SURVIVING_SPOUSE": {
            "2026-01-01.2026-12-31": 767125,
            "2027-01-01.2027-12-31": 782650,
            "2028-01-01.2028-12-31": 797775,
            "2029-01-01.2029-12-31": 812875,
            "2030-01-01.2030-12-31": 828850,
            "2031-01-01.2031-12-31": 845250,
            "2032-01-01.2032-12-31": 862100,
            "2033-01-01.2033-12-31": 879350,
            "2034-01-01.2034-12-31": 897050,
            "2035-01-01.2035-12-31": 915200
        },
        "gov.irs.income.exemption.amount": {
            "2026-01-01.2100-12-31": 0
        }
    }


In [6]:

def calculate_gini(income):
    sorted_income = np.sort(income)
    n = len(income)
    index = np.arange(1, n + 1)
    return ((2 * index - n - 1) * sorted_income).sum() / (n * sorted_income.sum())

In [7]:
def calculate_metrics(simulation, year):
    """Calculate raw metrics for a given simulation"""
    # Income
    income = simulation.calculate("household_net_income", period=year)
    
    # Poverty
    poverty = simulation.calculate("in_poverty", map_to="person", period=year)
    poverty_rate = poverty.mean()
    
    # Filter out negative decile values
    decile = simulation.calculate("household_income_decile", period=year)
    income_filtered = income[decile >= 1]
    
    # Calculate average income by decile
    avg_income_by_decile = (
        income_filtered.groupby(decile[decile >= 1]).sum()
        / income_filtered.groupby(decile[decile >= 1]).count()
    )
    
    # Convert decile values to dictionary format
    decile_values = {
        f"income_p{i}0_{i+1}0": avg_income_by_decile[i + 1]
        for i in range(10)
    }
    
    return {
        "total_income": income.sum(),
        "poverty_rate": poverty_rate,
        "inequality": calculate_gini(income_filtered),
        **decile_values
    }


In [8]:
# Generate and sort combinations
combinations = generate_policy_combinations()
# Sort combinations so that behavioral=False comes first, then behavioral=True
combinations.sort(key=lambda x: (x[2], x[0]["name"], x[1]["name"]))

In [9]:
EXPECTED_COLUMNS = [
    "scenario",
    "total_income",
    "poverty_rate",
    "inequality",
] + [f"income_p{i}0_{i+1}0" for i in range(10)]

# Create or load existing results
results_file = "../data/raw_metrics.csv"
try:
    existing_results = pd.read_csv(results_file)
    # Ensure all expected columns exist
    for col in EXPECTED_COLUMNS:
        if col not in existing_results.columns:
            existing_results[col] = None
    # Reorder columns to match expected order
    existing_results = existing_results[EXPECTED_COLUMNS]
    completed_scenarios = set(existing_results["scenario"].tolist())
    print(f"Found {len(completed_scenarios)} completed scenarios")
except (FileNotFoundError, pd.errors.EmptyDataError, pd.errors.ParserError):
    existing_results = pd.DataFrame(columns=EXPECTED_COLUMNS)
    completed_scenarios = set()
    print("Starting fresh with no existing results")

# First, calculate baseline scenario if not already done
if "baseline" not in completed_scenarios:
    print("Calculating baseline scenario")
    baseline = Microsimulation(dataset='enhanced_cps_2024')
    baseline_metrics = calculate_metrics(baseline, "2026")
    baseline_row = pd.DataFrame([{
        "scenario": "baseline",
        **baseline_metrics
    }])[EXPECTED_COLUMNS]
    existing_results = pd.concat([existing_results, baseline_row], ignore_index=True)
    existing_results.to_csv(results_file, index=False)
    completed_scenarios.add("baseline")

# Then calculate TCJA extension baseline if not already done
if "tcja_extension_baseline" not in completed_scenarios:
    print("Calculating TCJA extension baseline")
    tcja_reform = Reform.from_dict(get_tcja_extension_reform(), country_id="us")
    tcja_baseline = Microsimulation(reform=tcja_reform, dataset='enhanced_cps_2024')
    tcja_metrics = calculate_metrics(tcja_baseline, "2026")
    tcja_row = pd.DataFrame([{
        "scenario": "tcja_extension_baseline",
        **tcja_metrics
    }])[EXPECTED_COLUMNS]
    existing_results = pd.concat([existing_results, tcja_row], ignore_index=True)
    existing_results.to_csv(results_file, index=False)
    completed_scenarios.add("tcja_extension_baseline")

# Generate combinations and split into non-behavioral and behavioral
combinations = generate_policy_combinations()
non_behavioral_combinations = [(s, a, b) for s, a, b in combinations if not b]
behavioral_combinations = [(s, a, b) for s, a, b in combinations if b]

print(f"Processing {len(non_behavioral_combinations)} non-behavioral combinations first")
print(f"Then will process {len(behavioral_combinations)} behavioral combinations")

# Process non-behavioral combinations first
for salt_config, amt_config, behavioral in non_behavioral_combinations:
    scenario_name = f"salt_{salt_config['name']}_amt_{amt_config['name']}_behavioral_responses{'_yes' if behavioral else '_no'}"
    
    if scenario_name in completed_scenarios:
        print(f"Skipping {scenario_name} - already calculated")
        continue
    
    print(f"Calculating {scenario_name}")
    
    try:
        reform_dict = create_reform_dict(salt_config, amt_config, behavioral)
        reform = Reform.from_dict(reform_dict, country_id="us")
        reformed = Microsimulation(reform=reform, dataset='enhanced_cps_2024')
        metrics = calculate_metrics(reformed, "2026")
        
        new_row = pd.DataFrame([{
            "scenario": scenario_name,
            **metrics
        }])[EXPECTED_COLUMNS]
        
        existing_results = pd.concat([existing_results, new_row], ignore_index=True)
        existing_results.to_csv(results_file, index=False)
        completed_scenarios.add(scenario_name)
        
        print(f"Completed and saved {scenario_name}")
        
    except Exception as e:
        print(f"Error calculating {scenario_name}: {str(e)}")
        with open("../data/error_log.txt", "a") as f:
            f.write(f"\nError on {scenario_name}: {str(e)}")
        continue

print("Completed all non-behavioral combinations")
print("Starting behavioral combinations")

# Then process behavioral combinations
for salt_config, amt_config, behavioral in behavioral_combinations:
    scenario_name = f"salt_{salt_config['name']}_amt_{amt_config['name']}_behavioral_responses{'_yes' if behavioral else '_no'}"
    
    if scenario_name in completed_scenarios:
        print(f"Skipping {scenario_name} - already calculated")
        continue
    
    print(f"Calculating {scenario_name}")
    
    try:
        reform_dict = create_reform_dict(salt_config, amt_config, behavioral)
        reform = Reform.from_dict(reform_dict, country_id="us")
        reformed = Microsimulation(reform=reform, dataset='enhanced_cps_2024')
        metrics = calculate_metrics(reformed, "2026")
        
        new_row = pd.DataFrame([{
            "scenario": scenario_name,
            **metrics
        }])[EXPECTED_COLUMNS]
        
        existing_results = pd.concat([existing_results, new_row], ignore_index=True)
        existing_results.to_csv(results_file, index=False)
        completed_scenarios.add(scenario_name)
        
        print(f"Completed and saved {scenario_name}")
        
    except Exception as e:
        print(f"Error calculating {scenario_name}: {str(e)}")
        with open("../data/error_log.txt", "a") as f:
            f.write(f"\nError on {scenario_name}: {str(e)}")
        continue

print(f"Completed {len(completed_scenarios)} out of {len(combinations) + 2} possible scenarios")

Found 50 completed scenarios
Processing 30 non-behavioral combinations first
Then will process 30 behavioral combinations
Skipping salt_0_cap_amt_repealed_behavioral_responses_no - already calculated
Calculating salt_0_cap_amt_pre_tcja_ex_pre_tcja_po_behavioral_responses_no
Completed and saved salt_0_cap_amt_pre_tcja_ex_pre_tcja_po_behavioral_responses_no
Skipping salt_0_cap_amt_pre_tcja_ex_tcja_po_behavioral_responses_no - already calculated
Skipping salt_0_cap_amt_tcja_ex_pre_tcja_po_behavioral_responses_no - already calculated
Skipping salt_0_cap_amt_tcja_both_behavioral_responses_no - already calculated
Skipping salt_tcja_base_amt_repealed_behavioral_responses_no - already calculated
Calculating salt_tcja_base_amt_pre_tcja_ex_pre_tcja_po_behavioral_responses_no
Completed and saved salt_tcja_base_amt_pre_tcja_ex_pre_tcja_po_behavioral_responses_no
Skipping salt_tcja_base_amt_pre_tcja_ex_tcja_po_behavioral_responses_no - already calculated
Skipping salt_tcja_base_amt_tcja_ex_pre_tcja

In [10]:
print(f"Completed {len(completed_scenarios)} out of {len(combinations) + 2} possible scenarios")

Completed 62 out of 62 possible scenarios


In [13]:
def add_impacts_to_single_year():
    """
    Calculate impacts by comparing each scenario to both baselines 
    (Current Law and Current Policy) and add results to single_year_impacts.csv
    """
    # Read the raw metrics
    raw_metrics = pd.read_csv("../data/raw_metrics.csv")
    
    # Get both baseline values
    current_law_baseline = raw_metrics[raw_metrics["scenario"] == "baseline"].iloc[0]
    current_policy_baseline = raw_metrics[raw_metrics["scenario"] == "tcja_extension_baseline"].iloc[0]
    
    # Initialize results DataFrame
    impacts = []
    
    # Process each non-baseline scenario twice - once for each baseline
    for _, row in raw_metrics[~raw_metrics["scenario"].isin(["baseline", "tcja_extension_baseline"])].iterrows():
        original_scenario = row["scenario"]
        
        # Calculate impacts against Current Law baseline
        current_law_impact = {
            "scenario": f"{original_scenario}_vs_current_law",
            "baseline": "Current Law",
            "total_income_impact": current_law_baseline["total_income"] - row["total_income"],
            "poverty_rate_impact": row["poverty_rate"] - current_law_baseline["poverty_rate"],
            "inequality_impact": row["inequality"] - current_law_baseline["inequality"]
        }
        
        # Add decile impacts for Current Law comparison
        for i in range(10):
            col = f"income_p{i}0_{i+1}0"
            current_law_impact[f"{col}_impact"] = row[col] - current_law_baseline[col]
        
        impacts.append(current_law_impact)
        
        # Calculate impacts against Current Policy baseline
        current_policy_impact = {
            "reform": f"{original_scenario}_vs_current_policy",
            "baseline": "Current Policy",
            "total_income_impact": current_policy_baseline["total_income"] - row["total_income"],
            "poverty_rate_impact": row["poverty_rate"] - current_policy_baseline["poverty_rate"],
            "inequality_impact": row["inequality"] - current_policy_baseline["inequality"]
        }
        
        # Add decile impacts for Current Policy comparison
        for i in range(10):
            col = f"income_p{i}0_{i+1}0"
            current_policy_impact[f"{col}_impact"] = row[col] - current_policy_baseline[col]
        
        impacts.append(current_policy_impact)
    
    # Convert to DataFrame
    impacts_df = pd.DataFrame(impacts)
    
    # Save to CSV
    impacts_df.to_csv("../data/single_year_impacts.csv", index=False)
    print(f"Saved impacts for {len(impacts)} scenarios (including both baseline comparisons)")
    
    return impacts_df

# Execute the function
impacts = add_impacts_to_single_year()

Saved impacts for 120 scenarios (including both baseline comparisons)
